Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [1]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
#hyper parameters:
BATCH_SIZE = 8
NUM_EPOCHS = 20

### My attempt at a ResNet Network Model

In [70]:
class ResidualBlock(tf.keras.layers.Layer):
    ''' Res Block for an attempt at a ResNet architecture clone'''
    def __init__(self, filters, downsample=False):
        super(ResidualBlock, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        self.filters = filters
        self.downsample = downsample

        # downsample will halve the image size
        strides = 2 if downsample else 1

        self.conv1 = tf.keras.layers.Conv2D(filters, 3, strides=strides, padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-4))
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()

        self.conv2 = tf.keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-4))
        self.bn2 = tf.keras.layers.BatchNormalization()

        # Optional downsampling for the shortcut path
        if downsample:
            self.downsample_conv = tf.keras.layers.Conv2D(filters, 1, strides=2, padding='same', use_bias=False)
            self.downsample_bn = tf.keras.layers.BatchNormalization()
        else:
            self.downsample_conv = None
            
    def call(self, inputs, training=False):
        shortcut = inputs

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        if self.downsample_conv:
            shortcut = self.downsample_conv(shortcut)
            shortcut = self.downsample_bn(shortcut, training=training)

        x = tf.keras.layers.add([x, shortcut])
        return self.relu(x)
        

class MockResNet(tf.keras.Model):
    def __init__(self):
        super(MockResNet, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [
            tf.keras.Input(shape=(300, 300, 3)),
            #data_augmentation,
            
            tf.keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
            
            tf.keras.layers.Dropout(0.4),
            ResidualBlock(64),
            tf.keras.layers.Dropout(0.4),
            ResidualBlock(64),

            ResidualBlock(128, downsample=True),
            tf.keras.layers.Dropout(0.4),
            ResidualBlock(128),
            tf.keras.layers.Dropout(0.4),

            ResidualBlock(256, downsample=True),
            tf.keras.layers.Dropout(0.4),
            # ResidualBlock(256),
            # tf.keras.layers.Dropout(0.4),

            ResidualBlock(512, downsample=True),
            tf.keras.layers.Dropout(0.4),
            # ResidualBlock(512),
            # tf.keras.layers.Dropout(0.3),

            tf.keras.layers.GlobalAveragePooling2D(),
            
            #tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(52, activation='softmax')
        ]
        self.sequential = tf.keras.Sequential(self.architecture)
    
    def call(self, inputs, training=False):
        return self.sequential(inputs, training=training)
        
    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Traditional CNN Model

In [17]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,
                             
                tf.keras.layers.Conv2D(32, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Flatten(),                                   # Flatten to vector
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(52, activation='softmax')
                ]
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Dataset Loading

In [13]:
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)
    
new_data = {}
for key, inner_dict in data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['class_label']
    new_data[img_path] = value

In [72]:
dataset = tf.data.Dataset.from_tensor_slices((new_data.keys(), new_data.values()))

def load_train_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [59]:
# this cell it optional
# this checks that the datasets  is balanced
import collections
v_counts = collections.Counter()
t_counts = collections.Counter()


for _, label in val_dataset:
    label = label.numpy().tolist() 
    v_counts.update(label)

for _, label in train_dataset:
    label = label.numpy().tolist() 
    t_counts.update(label)
    
print("Label counts in validation dataset:")
for i in range(52):
    print(f"Label {i}: {v_counts[i]}, {t_counts[i]}")

Label counts in validation dataset:
Label 0: 29, 104
Label 1: 22, 110
Label 2: 35, 108
Label 3: 27, 112
Label 4: 35, 101
Label 5: 27, 108
Label 6: 28, 113
Label 7: 25, 104
Label 8: 29, 109
Label 9: 28, 109
Label 10: 34, 109
Label 11: 23, 94
Label 12: 28, 110
Label 13: 27, 114
Label 14: 18, 113
Label 15: 25, 109
Label 16: 24, 109
Label 17: 21, 98
Label 18: 23, 111
Label 19: 22, 102
Label 20: 33, 110
Label 21: 24, 101
Label 22: 26, 104
Label 23: 33, 111
Label 24: 38, 99
Label 25: 21, 107
Label 26: 26, 101
Label 27: 21, 105
Label 28: 9, 112
Label 29: 29, 105
Label 30: 18, 117
Label 31: 25, 108
Label 32: 28, 105
Label 33: 30, 112
Label 34: 26, 120
Label 35: 20, 111
Label 36: 22, 110
Label 37: 29, 109
Label 38: 22, 102
Label 39: 18, 114
Label 40: 29, 100
Label 41: 28, 107
Label 42: 26, 114
Label 43: 34, 113
Label 44: 25, 104
Label 45: 38, 100
Label 46: 26, 109
Label 47: 31, 109
Label 48: 32, 108
Label 49: 24, 108
Label 50: 28, 108
Label 51: 35, 110


2025-04-24 18:29:25.803133: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Model Loading and Training

In [74]:
#model = CardPredictor()
model = MockResNet()
#model = tf.keras.models.load_model("model_weights_89.keras")
model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])

In [75]:
model.fit(train_dataset, 
          validation_data=val_dataset, 
          epochs=NUM_EPOCHS, 
          verbose=1)

Epoch 1/10


ValueError: Shape must be rank 1 but is rank 0 for '{{node in_top_k/InTopKV2}} = InTopKV2[T=DT_INT32](mock_res_net_8_1/sequential_13_1/dense_16_1/Softmax, ArgMax, in_top_k/InTopKV2/k)' with input shapes: [?,52], [], [].

In [49]:
model.save('model_weights_89.keras')  # can also use .h5 extension if preferred